In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import platform

# 운영체제별 폰트 자동 설정
if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
else:
    plt.rc('font', family='DejaVu Sans')

plt.rcParams['axes.unicode_minus'] = False  # 마이너스 깨짐 방지

# 경로 설정
current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(current_dir))
DATA_DIR = os.path.join(project_root, 'data', 'processed')
DATA_DIR_MODELING = os.path.join(project_root, 'data', 'modeling')

# 1. 데이터 로딩 (기존 충전소 데이터 로딩 제거)
grids = pd.read_csv(os.path.join(DATA_DIR, 'grid_features_with_prediction_all.csv'))
mclp_sites = pd.read_csv(os.path.join(DATA_DIR_MODELING, 'mclp_selected_optimal.csv'))

# 2. 커버 수요/커버율 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 6371 * 2 * np.arcsin(np.sqrt(a))

def get_coverage(grids, selected_sites, R=1000):
    def is_covered(row):
        return any(
            haversine(row['center_lat'], row['center_lon'], s['center_lat'], s['center_lon']) <= R/1000
            for _, s in selected_sites.iterrows()
        )
    grids['covered'] = grids.apply(is_covered, axis=1)
    covered_demand = grids.loc[grids['covered'], 'demand_score'].sum()
    total_demand = grids['demand_score'].sum()
    coverage_rate = covered_demand / total_demand * 100
    return covered_demand, coverage_rate

# 3. p 값 입력 받기 (최대 60)
p_values = input("설치 개수(p)를 쉼표로 구분하여 입력하세요 (최대 60, 예: 10,20,30): ")
p_list = [int(p.strip()) for p in p_values.split(',')]
p_list = [min(p, 60) for p in p_list]
print(f"입력된 p 값(최대 60): {p_list}")

# 4. 시나리오별 데이터 생성 (기존 충전소 시나리오 제거)
scenario_results = []

# (1) 랜덤 시나리오
for p in p_list:
    random_sites = grids.sample(n=p, random_state=42)[['center_lat', 'center_lon']]
    covered_demand, coverage_rate = get_coverage(grids, random_sites)
    scenario_results.append({
        '시나리오': '랜덤',
        'p': p,
        '커버 수요': covered_demand,
        '커버율(%)': coverage_rate
    })

# (2) MCLP 시나리오
mclp_selected = mclp_sites[mclp_sites['selected'] == 1]
for p in p_list:
    selected = mclp_selected.head(min(p, len(mclp_selected)))
    covered_demand, coverage_rate = get_coverage(grids, selected)
    scenario_results.append({
        '시나리오': 'MCLP',
        'p': len(selected),
        '커버 수요': covered_demand,
        '커버율(%)': coverage_rate
    })

# 5. 결과 테이블
results_df = pd.DataFrame(scenario_results)
print("\n✅ 시나리오별 결과")
print(results_df)

# 6. 막대 그래프 (기존 충전소 기준선 제거)
plt.figure(figsize=(10, 6))
palette = {'랜덤': 'skyblue', 'MCLP': 'orange'}
sns.barplot(
    data=results_df,
    x='p', y='커버율(%)', hue='시나리오', palette=palette
)

plt.title('설치 개수(p)별 커버율: 랜덤 vs MCLP')
plt.xlabel('설치 개수(p)')
plt.ylabel('커버율(%)')
plt.legend(title='시나리오')
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'd:\\projects\\2025-1\\빅데이터프로그래밍\\data\\processed\\grid_features_with_prediction_all.csv'